In [1]:
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, regularizers, optimizers, initializers
from tensorflow.keras.utils import to_categorical

2022-02-10 20:41:28.762941: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/x86_64-linux-gnu/gazebo-11/plugins:/opt/ros/foxy/opt/yaml_cpp_vendor/lib:/opt/ros/foxy/opt/rviz_ogre_vendor/lib:/opt/ros/foxy/lib/x86_64-linux-gnu:/opt/ros/foxy/lib
2022-02-10 20:41:28.762978: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [3]:
train_batches = [
    "./cifar-10-batches-py/data_batch_1",
    "./cifar-10-batches-py/data_batch_2",
    "./cifar-10-batches-py/data_batch_3",
    "./cifar-10-batches-py/data_batch_4",
    "./cifar-10-batches-py/data_batch_5"
]
test_batch = "./cifar-10-batches-py/test_batch"

In [6]:
for index, batch_file in enumerate(train_batches):
    batch_data = unpickle(batch_file)
    if index == 0:
        train_x = batch_data[b"data"]
        train_y = batch_data[b"labels"]
    else:
        train_x = np.concatenate((train_x, batch_data[b"data"]))
        train_y = np.concatenate((train_y, np.array(batch_data[b"labels"])))
test_data = unpickle(test_batch)
test_x, test_y = test_data[b"data"], np.array(test_data[b"labels"])

train_x = train_x.reshape((train_x.shape[0], 32, 32, 3))
test_x = test_x.reshape((test_x.shape[0], 32, 32, 3))
train_x, test_x = train_x.astype("float32"), test_x.astype("float32")
train_x, test_x = train_x / 255., test_x / 255.

train_y = to_categorical(train_y)
test_y = to_categorical(test_y)

print("Train dataset shape: ", train_x.shape)
print("Train labels shape: ", train_y.shape)
print("Test dataset shape: ", test_x.shape)
print("Test labels shape: ", test_y.shape)

Train dataset shape:  (50000, 32, 32, 3)
Train labels shape:  (50000, 10)
Test dataset shape:  (10000, 32, 32, 3)
Test labels shape:  (10000, 10)


In [7]:
model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='linear', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((3, 3)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((3, 3)))
model.add(layers.Dense(120, activation='linear'))
model.add(layers.Dense(60, activation='relu'))
model.add(layers.Dense(10))
model.summary(line_length = 100)

Model: "sequential"
____________________________________________________________________________________________________
Layer (type)                                 Output Shape                            Param #        
conv2d (Conv2D)                              (None, 30, 30, 64)                      1792           
____________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)                 (None, 10, 10, 64)                      0              
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                            (None, 8, 8, 128)                       73856          
____________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)               (None, 2, 2, 128)                       0              
_______________________________________________________________________

2022-02-10 20:43:14.051211: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/x86_64-linux-gnu/gazebo-11/plugins:/opt/ros/foxy/opt/yaml_cpp_vendor/lib:/opt/ros/foxy/opt/rviz_ogre_vendor/lib:/opt/ros/foxy/lib/x86_64-linux-gnu:/opt/ros/foxy/lib
2022-02-10 20:43:14.051261: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-10 20:43:14.051298: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (abel-Vostro-3405): /proc/driver/nvidia/version does not exist
2022-02-10 20:43:14.051807: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To ena

In [ ]:
model.compile(
    optimizer=optimizers.Adagrad(learning_rate=0.05),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])
model.fit(train_x, train_y, epochs=1, validation_data=(test_x, test_y), batch_size=15)
_, test_acc = model.evaluate(test_x, test_y, verbose = 2)
print("Result:", test_acc)